<h1>Federated Learning - GTEx_V8 Example</h1>
<h2>Populate remote PyGrid nodes with labeled tensors </h2>
In this notebook, we will populate our PyGrid nodes with labeled data so that it will be used later by people interested in train models.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

Components:
 - PyGrid Network (http://localhost:5000)
 - PyGrid Node h1 (http://localhost:3000)
 - PyGrid Node h2 (http://localhost:3001)
 
Code implementation for this notebook has been referred from <a href="https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/grid/federated_learning/mnist/Fed.Learning%20MNIST%20%5B%20Part-1%20%5D%20-%20Populate%20a%20Grid%20Network%20(%20Dataset%20).ipynb">Fed.Learning MNIST [ Part-1 ] - Populate a Grid Network ( Dataset )</a> tutorial

<h2>Import dependencies</h2>

In [3]:
import syft as sy

#########<syft==0.2.8>#######################
# # Dynamic FL -->
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient

# #Static FL -->
from syft.grid.clients.model_centric_fl_client import ModelCentricFLClient
#############################################

#########<syft==0.2.6>#######################
# Dynamic FL -->
# from syft.grid.clients.dynamic_fl_client import DynamicFLClient

#Static FL -->
# from syft.grid.clients.static_fl_client import StaticFLClient
#############################################

import torch
import pickle
import time
import numpy as np
import torchvision
from torchvision import datasets, transforms
# import tqdm
# from ipywidgets import IntProgress

In [4]:
sy.version.__version__

'0.2.8'

<h2>Setup config</h2>
Init hook, connect with grid nodes, etc...

In [5]:
hook = sy.TorchHook(torch)

# Connect directly to grid nodes
nodes = ["ws://0.0.0.0:3000/",
         "ws://0.0.0.0:3001/"]

compute_nodes = []
for node in nodes:
    # For syft 0.2.8 --> replace DynamicFLClient with DataCentricFLClient
    compute_nodes.append( DataCentricFLClient(hook, node) )

In [6]:
compute_nodes

[<Federated Worker id:h1>, <Federated Worker id:h2>]

## 1 - Load Dataset

The code below will load GTEx data samples.

In [7]:
DATA_PATH = 'data/balanced/numpy_files/'
shared_x1 = np.load(DATA_PATH + 'shared_x1.npy') # First chunk of dataset 
shared_x2 = np.load(DATA_PATH + 'shared_x2.npy') # Second chunk of dataset 

shared_y1 = np.load(DATA_PATH + 'shared_y1.npy') # First chunk of labels 
shared_y2 = np.load(DATA_PATH + 'shared_y2.npy') # Second chunk of labels 

# Convert numpy array to torch tensors -->
shared_x1 = torch.from_numpy(shared_x1)
shared_x2 = torch.from_numpy(shared_x2)
shared_y1 = torch.from_numpy(shared_y1)
shared_y2 = torch.from_numpy(shared_y2)

shared_x1 = torch.tensor(shared_x1, dtype=torch.float32)
shared_x2 = torch.tensor(shared_x2, dtype=torch.float32)
shared_y1 = torch.tensor(shared_y1, dtype=torch.int64)
shared_y2 = torch.tensor(shared_y2, dtype=torch.int64)

datasets  = [shared_x1, shared_x2]
labels = [shared_y1, shared_y2]

/Users/pankajverma/anaconda3/envs/pysyft_v028/lib/python3.6/site-packages/syft/frameworks/torch/hook/hook.py:560: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


# Below using centralized way (full data) --->

In [6]:
# Concatenate 
X = torch.cat((shared_x1, shared_x2), dim=0)
Y = torch.cat((shared_y1, shared_y2), dim=0)

from torch import nn, optim
import torch.nn.functional as F

# TODO: Define your network architecture here
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(18420, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 6)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        
        return x
    
# Create the network, define the criterion and optimizer
model = Classifier()
# criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
    
epochs = 20

for e in range(epochs):

    epoch_loss = 0.0
    epoch_acc = 0.0
    optimizer.zero_grad()
    
    pred = model(X)
    loss = F.cross_entropy(pred, Y)

    loss.backward()
    optimizer.step()
    
    # statistics
    #prob = F.softmax(pred, dim=1)
    top1 = torch.argmax(pred, dim=1)
    ncorrect = torch.sum(top1 == Y)

    epoch_loss += loss.item()
    
    epoch_acc += ncorrect.item()

    epoch_loss /= Y.shape[0]
    epoch_acc /= Y.shape[0]

    print(f"Epoch: {e}",f"Training loss: {epoch_loss}", f" | Training Accuracy: {epoch_acc}")

Epoch: 0 Training loss: 0.001493217945098877  | Training Accuracy: 0.16666666666666666
Epoch: 1 Training loss: 0.0014930233359336854  | Training Accuracy: 0.1675
Epoch: 2 Training loss: 0.0014896153410275776  | Training Accuracy: 0.24666666666666667
Epoch: 3 Training loss: 0.001487082044283549  | Training Accuracy: 0.1975
Epoch: 4 Training loss: 0.0014871734380722047  | Training Accuracy: 0.2075
Epoch: 5 Training loss: 0.00148458997408549  | Training Accuracy: 0.19833333333333333
Epoch: 6 Training loss: 0.0014761630694071451  | Training Accuracy: 0.25
Epoch: 7 Training loss: 0.001480031708876292  | Training Accuracy: 0.20916666666666667
Epoch: 8 Training loss: 0.0014663223425547282  | Training Accuracy: 0.25
Epoch: 9 Training loss: 0.0014640167355537415  | Training Accuracy: 0.245
Epoch: 10 Training loss: 0.0014551534255345662  | Training Accuracy: 0.2708333333333333
Epoch: 11 Training loss: 0.0014517199993133545  | Training Accuracy: 0.2825
Epoch: 12 Training loss: 0.00144642472267150

<h2>2 - Tagging tensors</h2>
The code below will add a tag (of your choice) to the data that will be sent to grid nodes. This tag is important as the network will need it to retrieve this data later.

In [8]:
tag_input = []
tag_label = []

for i in range(len(compute_nodes)):
    tag_input.append(datasets[i].tag("#X", "#gtex_v8", "#dataset","#balanced").describe("The input datapoints to the GTEx_V8 dataset."))
    tag_label.append(labels[i].tag("#Y", "#gtex_v8", "#dataset","#balanced").describe("The input labels to the GTEx_V8 dataset."))

<h2> 3 - Sending our tensors to grid nodes</h2>

In [9]:
shared_x1 = tag_input[0].send(compute_nodes[0]) # First chunk of dataset to h1
shared_x2 = tag_input[1].send(compute_nodes[1]) # Second chunk of dataset to h2

shared_y1 = tag_label[0].send(compute_nodes[0]) # First chunk of labels to h1
shared_y2 = tag_label[1].send(compute_nodes[1]) # Second chunk of labels to h2

In [10]:
print("X tensor pointers: ", shared_x1, shared_x2)
print("Y tensor pointers: ", shared_y1, shared_y2)

X tensor pointers:  (Wrapper)>[PointerTensor | me:23284623177 -> h1:84769424305]
	Tags: #gtex_v8 #dataset #balanced #X 
	Shape: torch.Size([600, 18420])
	Description: The input datapoints to the GTEx_V8 dataset.... (Wrapper)>[PointerTensor | me:19362357387 -> h2:87452156177]
	Tags: #gtex_v8 #dataset #balanced #X 
	Shape: torch.Size([600, 18420])
	Description: The input datapoints to the GTEx_V8 dataset....
Y tensor pointers:  (Wrapper)>[PointerTensor | me:21210979203 -> h1:4194035554]
	Tags: #gtex_v8 #dataset #balanced #Y 
	Shape: torch.Size([600])
	Description: The input labels to the GTEx_V8 dataset.... (Wrapper)>[PointerTensor | me:14991428073 -> h2:94399344004]
	Tags: #gtex_v8 #dataset #balanced #Y 
	Shape: torch.Size([600])
	Description: The input labels to the GTEx_V8 dataset....


<h2>Disconnect nodes</h2>

In [11]:
for i in range(len(compute_nodes)):
    compute_nodes[i].close()